In [ ]:
import sagemaker
import boto3

sm_session = sagemaker.Session()
role = sagemaker.get_execution_role()
region = sm_session.boto_region_name
account = sm_session.account_id()


container_name = "deepspeed-sm"
container_tag = "latest"
image = '{}.dkr.ecr.{}.amazonaws.com/{}:{}'.format(account, region, container_name, container_tag)

In [ ]:
!aws ecr get-login-password --region {region} | docker login --username AWS --password-stdin 763104351884.dkr.ecr.{region}.amazonaws.com
# loging to your private ECR
!aws ecr get-login-password --region {region} | docker login --username AWS --password-stdin {account}.dkr.ecr.{region}.amazonaws.com

In [85]:
! ./build_and_push.sh $container_name $container_tag Dockerfile

Working in region us-east-1
WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
Sending build context to Docker daemon  357.1MB
Step 1/35 : ARG REGION
Step 2/35 : FROM 763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-training:1.6.0-gpu-py36-cu101-ubuntu16.04
 ---> 0d7223a296dd
Step 3/35 : ENV STAGE_DIR=/tmp
 ---> Using cache
 ---> 0fc2a869e3c1
Step 4/35 : RUN mkdir -p ${STAGE_DIR}
 ---> Using cache
 ---> 89ec507ea106
Step 5/35 : RUN apt-get update &&     apt-get install -y --no-install-recommends         software-properties-common autotools-dev         nfs-common pdsh g++ gcc tmux less unzip         htop iftop iotop rsync iputils-ping         net-tools         sudo
 ---> Using cache
 ---> 96966dced7ae
Step 6/35 : RUN wget h

In [ ]:
! docker images ls

In [ ]:
! pip install 'sagemaker[local]' --upgrade

In [ ]:
from sagemaker.local import LocalSession

# Configure our local training session
sagemaker_local_session = LocalSession()
sagemaker_local_session.config = {'local': {'local_code': True}}

In [ ]:
! pygmentize training_container/cifar_ds_config.json

In [ ]:
est = sagemaker.estimator.Estimator(image,
                                    role=role,
                                    instance_count=1,
                                    instance_type='local_gpu',
                                    hyperparameters = {
                                        "epochs" : 20,
                                        "deepspeed-config-file":"cifar_ds_config.json",
                                        "train-script":"train_cifar10.py"
                                    },
                                    sagemaker_session=sagemaker_local_session
)

est.fit(inputs={"data":"file:///home/ec2-user/SageMaker/deepspeed-on-aws/data/"})

Building with native build. Learn about native build in Compose here: https://docs.docker.com/go/compose-native-build/
Creating ircskvc74c-algo-1-uyd8k ... 
Creating ircskvc74c-algo-1-uyd8k ... done
Attaching to ircskvc74c-algo-1-uyd8k
ircskvc74c-algo-1-uyd8k | 2021-02-15 20:01:25,428 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
ircskvc74c-algo-1-uyd8k | 2021-02-15 20:01:25,495 sagemaker-training-toolkit INFO     Failed to parse hyperparameter deepspeed-config-file value cifar_ds_config.json to Json.
ircskvc74c-algo-1-uyd8k | Returning the value itself
ircskvc74c-algo-1-uyd8k | 2021-02-15 20:01:25,496 sagemaker-training-toolkit INFO     Failed to parse hyperparameter train-script value train_cifar10.py to Json.
ircskvc74c-algo-1-uyd8k | Returning the value itself
ircskvc74c-algo-1-uyd8k | 2021-02-15 20:01:25,507 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
ircskvc74c-algo-1-uyd8k | 2021-02-15 20:0

# Remote start

In [81]:
est = sagemaker.estimator.Estimator(image,
                                    role=role,
                                    instance_count=2,
                                    instance_type='ml.p3.16xlarge',
                                    hyperparameters = {
                                        "epochs" : 20,
                                        "deepspeed-config-file":"cifar_ds_config.json"
                                    },
                                    sagemaker_session=sm_session
)

est.fit(inputs={"data":"s3://vadimd-cv-datasets/cifar-10-deepspeed/"})


2021-02-15 19:39:22 Starting - Starting the training job...
2021-02-15 19:39:45 Starting - Launching requested ML instancesProfilerReport-1613417961: InProgress
.........
2021-02-15 19:41:10 Starting - Preparing the instances for training.........
2021-02-15 19:42:47 Downloading - Downloading input data
2021-02-15 19:42:47 Training - Downloading the training image....................bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2021-02-15 19:46:04,468 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2021-02-15 19:46:04,535 sagemaker-training-toolkit INFO     Failed to parse hyperparameter deepspeed-config-file value cifar_ds_config.json to Json.
Returning the value itself
2021-02-15 19:46:04,546 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.

2021-02-15 19:46:23 Uploading - Uploading generated training model
2021-02-15 19:46:23 Completed 